In [1]:
from planet import api
import sys

## Use the [Planet API](https://developers.planet.com/docs/apis/data/) to pull Satellite Imagery

In [2]:
# used http://geojson.io to get the coordinates for SF Bay

geojson_geometry = {
  "type": "Polygon",
  "coordinates": [
    [
      [
              -122.38872528076173,
              37.82442958216432
            ],
            [
              -122.29534149169922,
              37.82442958216432
            ],
            [
              -122.29534149169922,
              37.915221780801055
            ],
            [
              -122.38872528076173,
              37.915221780801055
            ],
            [
              -122.38872528076173,
              37.82442958216432
            ]
    ]
  ]
}

In [13]:
# get images that overlap with our AOI 
geometry_filter = {
  "type": "GeometryFilter",
  "field_name": "geometry",
  "config": geojson_geometry
}

In [14]:
import os
import json
import requests
from requests.auth import HTTPBasicAuth

PLANET_API_KEY = '3f77623fe0684b5eb61d9622c5002be6'

# Using REOrthoTile as specified in the Planet API downloading docs with the Visual asset returns a colored image
item_type = "REOrthoTile"

# API request object, depending on the filter specifications, the returned images may return none if the conditions aren't met
search_request = {
  "item_types": [item_type], 
  "filter": combined_filter
}

# fire off the POST request
search_result = \
  requests.post(
    'https://api.planet.com/data/v1/quick-search',
    auth=HTTPBasicAuth(PLANET_API_KEY, ''),
    json=search_request)

In [15]:
# extract image IDs only
image_ids = [feature['id'] for feature in search_result.json()['features']]
print(image_ids)

['20200326_183130_1056517_RapidEye-1', '20200325_182412_1056517_RapidEye-1', '20200321_182514_1056517_RapidEye-4', '20200319_182709_1056517_RapidEye-3', '20200303_182256_1056517_RapidEye-2', '20200302_183517_1056517_RapidEye-1', '20200227_183232_1056517_RapidEye-4', '20200225_183148_1056517_RapidEye-3', '20200224_182304_1056517_RapidEye-3', '20200220_183526_1056517_RapidEye-1', '20200219_182621_1056517_RapidEye-1', '20200212_183739_1056517_RapidEye-2', '20200210_183846_1056517_RapidEye-1', '20200206_183221_1056517_RapidEye-4', '20200204_182923_1056517_RapidEye-3', '20200106_184206_1056517_RapidEye-2', '20191228_183956_1056517_RapidEye-2', '20191124_183848_1056517_RapidEye-2', '20191118_183257_1056517_RapidEye-3', '20191103_183312_1056517_RapidEye-3', '20191101_183541_1056517_RapidEye-2', '20181116_185852_1056517_RapidEye-4', '20191014_183807_1056517_RapidEye-4', '20190921_184225_1056517_RapidEye-2', '20190916_183527_1056517_RapidEye-3', '20190823_183027_1056517_RapidEye-4', '20190822_1

In [16]:
# Just grab the first image ID
id0 = image_ids[0]
id0_url = 'https://api.planet.com/data/v1/item-types/{}/items/{}/assets'.format(item_type, id0)

# Returns JSON metadata for assets in this ID. Learn more: planet.com/docs/reference/data-api/items-assets/#asset
result = \
  requests.get(
    id0_url,
    auth=HTTPBasicAuth(PLANET_API_KEY, '')
  )

# List of asset types available for this particular satellite image
print(result.json().keys())

dict_keys(['analytic', 'analytic_sr', 'analytic_xml', 'udm', 'visual', 'visual_xml'])


In [17]:
# This is "inactive" if the "analytic" asset has not yet been activated; otherwise 'active'
print(result.json()['visual']['status'])

inactive


In [18]:
# Parse out useful links
links = result.json()[u"visual"]["_links"]
self_link = links["_self"]
activation_link = links["activate"]

# Request activation of the 'analytic' asset:
activate_result = \
  requests.get(
    activation_link,
    auth=HTTPBasicAuth(PLANET_API_KEY, '')
  )

In [20]:
activation_status_result = \
  requests.get(
    self_link,
    auth=HTTPBasicAuth(PLANET_API_KEY, '')
  )
    
print(activation_status_result.json()["status"])

activating


In [29]:
print(activation_status_result.json()["status"])

activating


In [28]:
download_link = activation_status_result.json()['location']
print(download_link)
# click link to download

KeyError: 'location'